In [ ]:
#| default_exp interface_pymeshlab

In [2]:
#| export

from blender_tissue_cartography import io as tcio

In [3]:
#| export

import numpy as np
import os
from copy import deepcopy
import pymeshlab

In [7]:
import matplotlib.pyplot as plt
from importlib import reload

## Interfacing with `pymeshlab`

In this notebook, we define functions to convert our `ObjMesh` class to and from `pymeshlab`.

In [6]:
mesh_fname_data = "registration_example/Drosophila_CAAX-mCherry_mesh_remeshed.obj"
mesh_fname_ref = "registration_example/Drosophila_reference_preregistered.obj"